# Simulación Monte Carlo con Clasificación Interactiva y Visualización con ECharts

## Objetivo de la Actividad

Esta actividad tiene como finalidad **enseñar el uso de simulaciones probabilísticas (Monte Carlo)** y la **clasificación de puntos mediante distancia euclidiana**. A través de un entorno visual e interactivo con Streamlit, los estudiantes o usuarios pueden experimentar cómo un sistema de clasificación simple puede tomar decisiones a partir de datos generados sintéticamente.

La app web construida permitirá:

- Generar dos clases de datos bidimensionales.
- Clasificar puntos nuevos según la distancia promedio a cada clase.
- Visualizar los resultados en gráficos interactivos.
- Descargar la clasificación resultante y revisar los datos simulados.

---

## ¿Qué es la Simulación Monte Carlo?

Es una técnica estadística que permite modelar fenómenos inciertos repitiendo procesos aleatorios muchas veces. En este caso, usamos distribuciones normales (Gauss) para generar puntos aleatorios en el plano y simular dos "clases".

---

## ¿Por qué usar la **distancia euclidiana**?

Porque es una forma intuitiva y simple de medir qué tan "cerca" está un punto nuevo de cada grupo. Esta idea es la base de muchos clasificadores, como el algoritmo de los **k vecinos más cercanos (KNN)**.

---

## Herramientas y librerías utilizadas

- **Streamlit**: Crea apps web interactivas en Python con solo escribir código de script.  
  👉 Documentación: https://docs.streamlit.io

- **NumPy**: Para crear datos simulados y manejar operaciones vectorizadas.  
  👉 https://numpy.org/doc/

- **Pandas**: Para organizar los datos en DataFrames y exportarlos.  
  👉 https://pandas.pydata.org/docs/

- **Streamlit-ECharts**: Permite insertar gráficos dinámicos y personalizados usando [Apache ECharts](https://echarts.apache.org).  
  👉 https://github.com/andfanilo/streamlit-echarts

---

## Componentes clave de la App

### 1. `st.sidebar` – Parámetros ajustables
La barra lateral permite al usuario:
- Cambiar el número de puntos generados.
- Controlar la dispersión (std) de las clases.
- Fijar centros personalizados.
- Introducir sus propios puntos para clasificar.
  
```
# ----------------------- PARÁMETROS ----------------------- #
st.sidebar.header("Parámetros de la Simulación")

n_samples = st.sidebar.slider("Número de muestras por clase", 100, 5000, 1000)
std_dev = st.sidebar.slider("Dispersión (std dev)", 0.5, 3.0, 1.2)
seed = st.sidebar.number_input("Seed aleatoria", value=42, step=1)
np.random.seed(seed)

st.sidebar.markdown("### Centro Clase 0")
class_0_x = st.sidebar.number_input("X Clase 0", value=2.0)
class_0_y = st.sidebar.number_input("Y Clase 0", value=2.0)

st.sidebar.markdown("### Centro Clase 1")
class_1_x = st.sidebar.number_input("X Clase 1", value=6.0)
class_1_y = st.sidebar.number_input("Y Clase 1", value=6.0)

```

### 2. `np.random.normal(...)` – Simulación de puntos
Se crean dos conjuntos de puntos usando una distribución normal, cada uno con su centro y desviación estándar definidos por el usuario.

```
# ----------------------- DATOS SIMULADOS ----------------------- #
center_0 = np.array([class_0_x, class_0_y])
center_1 = np.array([class_1_x, class_1_y])

class_0 = np.random.normal(loc=center_0, scale=std_dev, size=(n_samples, 2))
class_1 = np.random.normal(loc=center_1, scale=std_dev, size=(n_samples, 2))
```

### 3. `np.sqrt(np.sum(...)).mean()` – Clasificación
Calculamos la distancia euclidiana promedio de cada punto nuevo respecto a los puntos de cada clase.

```
# ----------------------- CLASIFICACIÓN ----------------------- #
results = []
for pt in new_points:
    pt_np = np.array(pt)
    dist_0 = np.sqrt(np.sum((class_0 - pt_np)**2, axis=1)).mean()
    dist_1 = np.sqrt(np.sum((class_1 - pt_np)**2, axis=1)).mean()

    pred = 0 if dist_0 < dist_1 else 1
    results.append({
        "X": pt[0],
        "Y": pt[1],
        "Distancia Clase 0": round(dist_0, 2),
        "Distancia Clase 1": round(dist_1, 2),
        "Clase Predicha": pred
    })
```

### 4. `streamlit_echarts.st_echarts(...)` – Visualización
Reemplazamos `matplotlib` por `ECharts` para una experiencia más interactiva: zoom, colores dinámicos, tooltips, etc.

```
# ----------------------- VISUALIZACIÓN CON ECHARTS ----------------------- #
st.subheader("Visualización Interactiva con ECharts")

# Separar puntos nuevos según su clase predicha
new_points_0 = [{"value": [r["X"], r["Y"]]} for r in results if r["Clase Predicha"] == 0]
new_points_1 = [{"value": [r["X"], r["Y"]]} for r in results if r["Clase Predicha"] == 1]

scatter_data_0 = [{"value": list(p)} for p in class_0]
scatter_data_1 = [{"value": list(p)} for p in class_1]

option = {
    "tooltip": {"trigger": "item"},
    "legend": {
        "data": ["Clase 0", "Clase 1", "Nuevos → Clase 0", "Nuevos → Clase 1"]
    },
    "xAxis": {"type": "value", "name": "X"},
    "yAxis": {"type": "value", "name": "Y"},
    "series": [
        {
            "name": "Clase 0",
            "type": "scatter",
            "data": scatter_data_0,
            "symbolSize": 6,
            "itemStyle": {"color": "blue", "opacity": 0.4}
        },
        {
            "name": "Clase 1",
            "type": "scatter",
            "data": scatter_data_1,
            "symbolSize": 6,
            "itemStyle": {"color": "red", "opacity": 0.4}
        },
        {
            "name": "Nuevos → Clase 0",
            "type": "scatter",
            "data": new_points_0,
            "symbolSize": 12,
            "symbol": "star",
            "itemStyle": {"color": "green", "opacity": 0.9}
        },
        {
            "name": "Nuevos → Clase 1",
            "type": "scatter",
            "data": new_points_1,
            "symbolSize": 12,
            "symbol": "star",
            "itemStyle": {"color": "purple", "opacity": 0.9}
        }
    ]
}

st_echarts(options=option, height="500px")
```

### 5. `st.dataframe(...)` – Revisión de resultados
Mostramos la tabla con cada punto clasificado y sus distancias.

```
# ----------------------- RESULTADOS ----------------------- #
st.subheader("Resultados de Clasificación")
df_results = pd.DataFrame(results)
st.dataframe(df_results)

csv = df_results.to_csv(index=False).encode("utf-8")
st.download_button("Descargar Resultados CSV", data=csv, file_name="resultados_clasificacion.csv", mime="text/csv")
```

### 6. `st.download_button(...)` – Exportación
Permite descargar los resultados en formato `.csv`.

```
# ----------------------- VER DATOS SIMULADOS ----------------------- #
st.subheader("Datos Simulados por Clase")
df_c0 = pd.DataFrame(class_0, columns=["x", "y"])
df_c0["Clase"] = 0
df_c1 = pd.DataFrame(class_1, columns=["x", "y"])
df_c1["Clase"] = 1
df_full = pd.concat([df_c0, df_c1], ignore_index=True)

if st.toggle("Mostrar datos simulados"):
    st.dataframe(df_full)
```

---

## ¿Por qué usar ECharts en lugar de Matplotlib?

- Más ligero y rápido para web.
- Permite interacción (hover, zoom).
- Estéticamente más moderno.
- No necesita generar imágenes estáticas.

---

## ¿Cómo correr esta app?

1. Crea un repositorio "Deploy" en tu cuenta GitHub
2. Crea tu archivo `streamlit_app.py`. Toma como referencia el archivo `streamlit_app_esqueleto.py`, añadiendo el código en los bloques correspondientes para hacerlo funcionar correctamente.
4. Crea un archivo `requirements.txt` con este contenido, sin espacios, tal cual aparecen:

    ```
    streamlit
    numpy
    pandas
    streamlit-echarts
    ```
    

5. Sube ambos archivos (`streamlit_app.py` y `requirements.txt`) a tu repositorio "Deploy" de GitHub.:

6. Vincula tu cuenta a Streamlit y crea la app correspondiente. Le darás clic a la opción `Deploy a public app from GitHub -> Deploy Now` y posteriormente ajusta las opciones de la siguiente manera:
   
    - Repositorio -> Your_Account/Deploy
    - Branch -> Main
    - Main file path -> streamlit_app.py
    - App URL (optional) -> dejamos el generado por default
    - Y finalmente, le damos la opción "Deploy" (ignora la sección de "Advanced Settings")

---

## Recursos útiles

- [Guía de Streamlit oficial](https://docs.streamlit.io/)
- [Galería de ECharts](https://echarts.apache.org/examples/en/index.html)
- [Ejemplos de `streamlit-echarts`](https://github.com/andfanilo/streamlit-echarts)

---

## Créditos

Este ejercicio fue creado para ilustrar conceptos fundamentales de clasificación, visualización interactiva y simulación estadística, con fines educativos.  
Autor: **Mauricio Rosales Rivera**  
Versión: 2025  
